In [4]:
from lxml import html as lhtml                         # Непосредственно для парсинга.
import requests                                        # Для получения html кода сайта.
from multiprocessing.dummy import Pool as ThreadPool   # Для реализации параллельной обкачки.
from math import ceil                                  # Для округления вверх.
from tqdm.notebook import tqdm                         # Для визуализации прогресса итерации по генератору.
import time                                            # Для ожидания окончания работы в  get_page().
from multiprocessing import Lock                       # Для логгирования.


import numpy as np  # Для дебага, чтобы получать произвольные url'ы.
import random       # Для дебага, чтобы получать произвольные url'ы.

import pandas as pd # Непосредственно для формирования *.csv файла.

In [5]:
with open('authors.txt', 'r') as f:
    authors_id = list(map(int, f.read().split('\n')[:-1]))

In [6]:
way = '//div[@class="rd-listing-product-item rd-listing-product-item_size- rd-listing-product-item_type-"]/a'

In [7]:
def get_page(url, n_attempts=5, t_sleep=1, **kwargs):
    r_get = requests.get(url, **kwargs)
    if not r_get.ok:
        time.sleep(t_sleep)
        if n_attempts > 0:
            return get_page(url, n_attempts-1, t_sleep, kwargs)
    return r_get   

In [8]:
def get_url(author):
    
    url_for_download = []

    page_url = "https://www.respublica.ru/authors/" + str(author)
    html = get_page(page_url)
    tree = lhtml.fromstring(html.text)
    all_page_nums = ceil(int(tree.xpath('//span[@class="rd-listing-count__total"]')[0].text) / 22)

    for page_num in range(all_page_nums):
        
        page_url = "https://www.respublica.ru/authors/" + str(author)
        html = get_page(page_url, params={'page': page_num, })
        tree = lhtml.fromstring(html.text)
        all_book_on_page = tree.xpath(way)

        for book in all_book_on_page:
            url_for_download.append("https://www.respublica.ru"+book.attrib['href'])
    return url_for_download

In [9]:
%%time

with ThreadPool(10) as pool:
    authors_book_urls = pool.map(get_url, authors_id)
pool.join()

url_history_for_download = [url for sub_list in authors_book_urls for url in sub_list]

CPU times: user 6.59 s, sys: 161 ms, total: 6.75 s
Wall time: 20.2 s


In [10]:
len(url_history_for_download)

2860

In [12]:
template = {
    "ID":       None,
    "URL":      None,
    "Название": None,
    "Автор":    None,
    "Превью":   None,
    "Изображение":   None,
    "Описание": None,
    "Цена":     None,
    "Цена (старая)": None,
    "В наличии": None,
    "Категория": None,
    "Число отзывов": 0,
    "Число оценок":  0,
    "Оценка":   None,
    "ISBN":     None,
    "Издательство":  None,
    "Серия":    None,
    "Обложка":  None,
    "Формат":   None,
    "Количество страниц":   None,
    "Год издания":   None,
    "Язык":     None,
    "Направление":   None,
    "Страна-производитель": None
}

In [14]:
indexes = np.random.randint(0,len(url_history_for_download), 100)

In [25]:
from multiprocessing import Pool, Lock, Value

mutex = Lock()
n_processed = Value('i', 0)

def get_site(url):
    
    field = template.copy()
    html = get_page(url)
    tree = lhtml.fromstring(html.text)
    
    field["ID"] = tree.xpath('//span[@itemprop="sku"]')[0].text
    field["URL"] = url
    field["Название"] = tree.xpath('//h1[@class="rd-page-product__title"]//text()')[0]
    field["Автор"] = tree.xpath('//a[@itemprop="brand"]//text()')[0]
    
    preview_path = tree.xpath('//a[@class="download-pdf"]')
    if bool(preview_path):
        field["Превью"] = "https://www.respublica.ru" + preview_path[0].attrib['href'] 
    
    field["Изображение"] = tree.xpath('//meta[@property="og:image"]')[0].attrib['content']
    field["Описание"] = ' '.join(tree.xpath('//div[@class="rd-page-product__desc-body"]//text()'))
    field["Цена"] = int(tree.xpath('//span[@class="num"]//text()')[0].replace(' ',''))
    
    old_price_path = tree.xpath('//div[@class="rd-page-product__price-old"]//span//text()')
    if bool(old_price_path):
        field["Цена (старая)"] = old_price_path[0]
    
    field["В наличии"] = bool(tree.xpath('//a[@class="rd-page-product__buy rd-page-product__buy_status_available"]'))
    
    categories = tree.xpath('//div[@class="rd-page-breadcrumbs rd-page-product__breadcrumbs"]//span[@itemprop="name"]//text()')
    field["Категория"] = '; '.join([item[1:] for item in categories])
    
    rating_block = tree.xpath('//span[@itemprop="aggregateRating"]')
    
    if bool(rating_block):
        
        field["Число отзывов"] = int(rating_block[0].xpath('//meta[@itemprop="reviewCount"]')[0].attrib['content'])
        field["Число оценок"] = int(rating_block[0].xpath('//meta[@itemprop="ratingCount"]')[0].attrib['content'])
        field["Оценка"] = float(rating_block[0].xpath('//meta[@itemprop="ratingValue"]')[0].attrib['content'])
    
    features = tree.xpath('//p[@class="rd-page-product__desc-param"]//text()')
    features = {features[i]: features[i+2] for i in range(0,len(features)-1,3)}
    keys = features.keys()
    
    names  = tree.xpath('//p[@class="rd-page-product__desc-param"]//span[@itemprop="name"]')
    values = tree.xpath('//p[@class="rd-page-product__desc-param"]//*[@itemprop="value"]')

    for idx in range(len(names)):
            field[names[idx].text] = values[idx].text
            
    return field

def get_site_wrapper(uid):
    result = get_site(uid)
    with mutex:
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return result

In [ ]:
%%time

with ThreadPool(100) as pool:
    all_sites = pool.map(get_site_wrapper, url_history_for_download)
pool.join()

2610 objects are processed...

In [ ]:
import pandas as pd
pd.options.display.max_columns=100
sites = pd.DataFrame(all_sites)
sites.head(10)

In [ ]:
sites.shape

In [ ]:
sites.isnull().sum()

In [ ]:
sites.to_csv('d.harazyan_hw_3.csv')

In [ ]:
# charact = ["ISBN", "Издательство", "Серия", "Обложка", "Формат", "Количество страниц",\
#            "Год издания", "Язык", "Вес, г", "Направление", "Возрастные ограничения", "Страна-производитель"]
# template = {key: None for key in charact}

# def get_site(url):
    
#     field = []
#     html = get_page(url)
#     tree = lhtml.fromstring(html.text)
#     tmp = tree.xpath('//div[@class="rd-page-breadcrumbs rd-page-product__breadcrumbs"]//span[@itemprop="name"]//text()')
    
#     field.append(url) # 0 - URL.
#     field.append([item[1:] for item in tmp]) #1 - Категория.
#     field.append(tree.xpath('//h1[@class="rd-page-product__title"]//text()')) #2 - Название.
#     field.append(tree.xpath('//a[@itemprop="brand"]//text()')[0]) # 3 - Автор.
#     field.append(tree.xpath('//span[@itemprop="sku"]')[0].text) # 4 - Артикул.
    
#     tmp = tree.xpath('//a[@class="download-pdf"]')
#     if bool(tmp):
#         tmp = tmp[0].attrib['href']
#         field.append("https://www.respublica.ru" + tmp) # 5 - Превью.
#     else:
#         field.append(None)
        
#     field.append(tree.xpath('//meta[@property="og:image"]')[0].attrib['content']) # 6 - Изображение.

#     total_mark     = len(tree.xpath('//div[@class="rd-page-product__rating-stars"]//div[@class="rd-rating-stars__star rd-rating-stars__star_fill"]'))
#     comment_number = len(tree.xpath('//div[@class="rd-product-reviews__item"]//div[@class="rd-product-reviews__item-body"]'))
#     mark_number    = len(tree.xpath('//div[@class="rd-product-reviews__item"]//div[@class="rd-rating-stars"]'))
#     field.append([total_mark, comment_number, mark_number]) # 7 - Оценка, отзывы, оценки.
    
#     field.append(int(tree.xpath('//span[@class="num"]//text()')[0].replace(' ',''))) # 8 - Цена.    
    
#     tmp = tree.xpath('//div[@class="rd-page-product__price-old"]//span//text()')
#     if bool(tmp):
#         field.append(tmp[0]) # 9 - старая Цена.
#     else:
#         field.append(None)
        
#     field.append(bool(tree.xpath('//a[@class="rd-page-product__buy rd-page-product__buy_status_available"]'))) # 10 - Наличие.
#     field.append(' '.join(tree.xpath('//div[@class="rd-page-product__desc-body"]//text()'))) # 11 - Описание.

#     tmp = tree.xpath('//p[@class="rd-page-product__desc-param"]//text()')
#     tmp = {tmp[i]: tmp[i+2] for i in range(0,len(tmp)-1,3)}
#     temp = template.copy()
#     for key in tmp.keys():
#         temp[key] = tmp[key]
#     field.append(temp) # 12 - Характеристики.    

#     return field